In [1]:
#!python -m spacy download el_core_news_sm
#!pip install pyarrow --upgrade

In [1]:
import spacy
import el_core_news_sm
import string
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
from spacy.lang.el import GreekLemmatizer

In [3]:
from spacy.lang.el import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

In [4]:
nlp = el_core_news_sm.load()

In [5]:
lemmatizer = GreekLemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)

In [6]:
p1 = re.compile('δεν απαντ.{1,3}\s{0,1}',re.IGNORECASE)
p2 = re.compile('\sδα\s',re.IGNORECASE)
p3 = re.compile('δε.{0,1}\s.{0,3}\s{0,1}βρ.{1,2}κ.\s{0,1}',re.IGNORECASE)

In [7]:
def loadStopWords():
    sWords = open('stopWords.txt','r',encoding='utf-8')
    sw = set(sWords.read().split('\n'))
    #sw = sw.remove('μη')
    sWords.close()
    return sw

In [8]:
def replaceTerm(text):
    text = text.replace('-banking','banking')
    text = text.replace('v banking','vbanking')
    text = text.replace('e banking','ebanking')
    text = text.replace('follow up','followup')
    text = text.replace('safe drive','safedrive')
    text = text.replace('safe pocket','safepocket')
    text = text.replace('sweet home','sweethome')
    text = text.replace('credit card','creditcard')
    text = text.replace('debit card','debitcard')
    text = text.replace('π/κ','πκ')
    text = text.replace('α/κ','ακ')
    text = text.replace('δ/α','δεναπαντα ')
    text = p1.sub(' δεναπαντα ',text)
    text = p2.sub(' δεναπαντα ',text)
    text = p3.sub(' δεντονβρηκα ',text)
    return text


In [9]:
#sw = nlp.Defaults.stop_words
#sw = sw|{'εχω','απο','ωστε'}
sw = loadStopWords()
def remove_ton(text):
    diction = {'ά':'α','έ':'ε','ί':'ι','ό':'ο','ώ':'ω','ύ':'υ'}
    for key in diction.keys():
        text = text.replace(key, diction[key])
    return text   
def clean_text(text):
     #text to string
    text = str(text).lower()
    text = replaceTerm(text)
   # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # lower text
    text = [remove_ton(x) for x in text]
    # remove stop words
    text = [x for x in text if x not in sw]
 
    #remove quotes
    text = [x.replace('quot;','').replace('&quot','') for x in text if x not in ['quot','amp']]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # remove amp & quot
    text = [x for x in text if x not in ['quot','amp']]
    # remove words with only one letter
    text = " ".join([t for t in text if len(t) > 1])
    # lemmatize text
    text = " ".join([lemmatizer(t.text,t.pos_)[0] for t in nlp(text)])
   
    return(text)

In [10]:
def correct(x,corDict):
    if x in corDict.keys():
        y = corDict[x]
    else:
        y = x
    return y    

In [11]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '6ed9d167-b2e6-41b8-9500-35e6df64d9dc'
resource_group = 'MLRG'
workspace_name = 'erbbimlws'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='LinkCommentsSample')
df = dataset.to_pandas_dataframe()

In [12]:
df = df[['CON_ROW_ID','CON_COMMENTS']]

In [13]:
#df.columns
df.head()
df.shape

(188199, 2)

In [14]:
df['tokenized'] = df['CON_COMMENTS'].apply(clean_text)

In [16]:
df.to_excel('LinkCommentssample.xlsx')
df.head(1000)



,CON_ROW_ID,CON_COMMENTS,tokenized
0,4829747.0,LISTA2,
1,54042370.0,Για την λήξη των 2 Value της ίδιας και των 3 V...,λήξη value value κορης μαριας
2,54042444.0,Για την λήξη των 3 Value της ίδιας και των 2 V...,λήξη value value μητερα
3,54154396.0,ΛΗΞΗ ΠΥΡΑΣΦΑΛΙΣΤΗΡΙΟΥ 13/02/2019 ΤΗΛ ΓΙΑ ΝΑ ΚΑ...,ληξη πυρασφαλιστηριος τηλ κανω προταση
4,55574534.0,ΤΟΠΟΘΕΤΗΣΗ ΣΕ Α/Κ ΚΑΙ ΕΚΤΑΚΤΗ ΚΑΤΑΒΟΛΗ ΣΤΟ ΤΑΚ...,τοποθετηση ακ εκτακτη καταβολη τακτο
...,...,...,...
995,62888661.0,οκ θα ερθει την επομενη εβδομαδα,οκ ερθω επομενη εβδομαδος
996,62888688.0,ΕΝΗΜΕΡΩΘΗΚΕ ΓΙΑ ΤΗΝ ΛΗΞΗ ΤΗΣ ΠΚ.ΕΙΝΑΙ ΣΤΗΝ ΕΠΑ...,ενημερωθηκε ληξη πκ.ειναι επαρχιο θελω τηλ ανα...
997,62888694.0,safe pocket ΝΕΧΤ ΤΙΜΕ,safepocket νεχτ τιμε
998,62888695.0,ΕΠΙΚΟΙΝΩΝΙΑ ΑΠΟ ΚΑΤΑΣΤΗΜΑ ΕΞΥΠΗΡΕΤΗΣΗΣ 164,επικοινωνιο καταστημας εξυπηρετηση


In [17]:
df = df.fillna('N/A')

In [29]:
df[df['tokenized'].str.contains(' χρονι ') ]#[~df['tokenized'].str.contains('banking') ]

,CON_ROW_ID,CON_COMMENTS,tokenized
23,6.27263e+07,ΤΙ ΕΓΙΝΕ ΜΕ ΤΗΝ ΑΓΟΡΑ ΚΑΤΟΙΚΙΑΣ;/ΜΕΧΡΙ ΤΗΝ ΠΑΡ...,εγινε αγορος κατοικιας;/μεχρι παρασκευη θεωρω ...
104,6.28578e+07,ΛΗΞΗ/ΕΝΗΜΕΡΩΘΗΚΕ ΓΙΑ ΠΡΟΘ ΜΕ ΕΥΡΩ ΕΠΙΣΤΡΟΦΗ ΚΑ...,ληξη / ενημερωθηκε προθ ευρω επιστροφη εκκινησ...
1137,6.56228e+07,ΕΝΗΜΕΡΩΘΗΚΕ ΓΙΑ ΛΗΞΗ Π/Κ. ΘΑ ΟΡΙΣΤΕΙ ΡΑΝΤΕΒΟΥ...,ενημερωθηκε ληξη πκ οριστω ραντεβο καταστημας ...
1359,6.57679e+07,ΕΝΗΜΕΡΩΣΗ ΓΙΑ ΕΞΑΣΦΑΛΙΖΩ ΣΥΣΤΗΜΑΤΙΚΗ ΕΠΕΝΔΥΣΗ ...,ενημερωση εξασφαλιζω συστηματικη επενδυση συμμ...
2318,6.62319e+07,ΣΥΝΕΧΙΖΕΙ ΝΑ ΕΙΝΑΙ ΔΥΣΚΟΛΗ Η ΚΑΤΑΣΤΑΣΗ ΜΕ ΤΗΝ...,συνεχιζω δυσκολη κατασταση εργασι εργαζομαι πρ...
...,...,...,...
138449,6.66614e+07,ΕΝΗΜΕΡΩΣΗ ΓΙΑ ΛΗΞΗ ΠΡΟΘΕΣΜΙΑΚΗΣ ΚΑΤΑΘΕΣΗΣ. ΘΑ ...,ενημερωση ληξη προθεσμιακης καταθεση προσελθω ...
138635,6.66618e+07,Δανειοδοτήθηκε μέσα στην χρονιά. Δεν υπάρχει κ...,δανειοδοτήθηκε χρονι υπαρχω αναγκη αυτή στιγμή...
138815,6.66623e+07,"ΔΕ ΒΡΕΘΗΚΑΜΕ ΚΑΘΟΛΟΥ ΦΕΤΟΣ ΛΟΓΩ ΤΟΥ ΚΟΡΩΝΟΙΟΥ,...","βρεθηκαμε κορωνοιος , με νεα χρονι καταφερω πω"
139114,6.66788e+07,Ευχήθηκα χρόνια πολλά,ευχήθηκα χρονι πολλας


In [18]:
tfidf = TfidfVectorizer(min_df = 1000,ngram_range = (1,3))

In [19]:
tfidf_result = tfidf.fit_transform(df['tokenized']).toarray()

In [20]:
tfidf_df = pd.DataFrame(tfidf_result, columns = tfidf.get_feature_names())

In [21]:
tfidf_df.columns = [str(x) for x in tfidf_df.columns]

In [22]:
df_f = pd.concat([df[['CON_ROW_ID']],tfidf_df],axis=1).melt(id_vars=['CON_ROW_ID'],value_vars = tfidf_df.columns).dropna()

In [23]:
df_f = df_f[df_f['value']>0]

In [24]:
df_f.shape

(268254, 3)

In [25]:
df_f['variable'].value_counts().to_excel('tokenlist.xlsx')

In [30]:
corDict = dict(pd.read_excel("corTokens.xls").to_dict("split")['data'])

In [31]:
df_f['token'] = df_f['variable'].apply(lambda x : correct(x,corDict))

In [32]:
df_f = df_f[df_f['token'] !='rmv']

In [33]:
df_f = df_f.fillna('N/A')

In [34]:
df_f = df_f.sort_values(['CON_ROW_ID','token'])

In [35]:
df_f['token_c'] = df_f['token']

In [36]:
n = df_f.shape[0]

In [37]:
df_f.head(100)

,CON_ROW_ID,variable,value,token,token_c
2959553,60373305.0,ασφαλει,0.808213,ασφαλεια,ασφαλεια
9442379,60373305.0,ληξη,0.588890,ληξη,ληξη
1127451,60995338.0,visa,0.751564,visa,visa
7187486,62513965.0,ευχες,0.713839,ευχες,ευχες
12401933,62513965.0,προγραμμας,0.700309,προγραμμα,προγραμμα
...,...,...,...,...,...
11697306,62735031.0,πκ,0.542472,προθεσμιακη κατάθεση,προθεσμιακη κατάθεση
9442411,62736090.0,ληξη,1.000000,ληξη,ληξη
1832138,62737213.0,ακ,0.296821,αμοιβαιο κεφαλαιο,αμοιβαιο κεφαλαιο
2254931,62737213.0,ανανεωση,0.270951,ανανεωση,ανανεωση


In [38]:
for i in range(0,n-1):
    if df_f['CON_ROW_ID'].iloc[i] == df_f['CON_ROW_ID'].iloc[i+1]:
        #print(df_f['token_c'].iloc[i],type(df_f['token_c'].iloc[i]),type(df_f['token_c'].iloc[i+1]),df_f['token_c'].iloc[i+1])
        if df_f['token_c'].iloc[i] in df_f['token_c'].iloc[i+1]:
            df_f.iloc[i,4] = df_f['token_c'].iloc[i+1]
        
            

In [39]:
df_f = df_f[['CON_ROW_ID','token_c']].drop_duplicates()

In [40]:
df_f.head()
df_f.shape

(235625, 2)

In [ ]:
df_f.to_csv('comments_tokens_2020_12.txt',sep ='\t',line_terminator='\r\n',index = False)

In [ ]:
#df_f.to_excel('vBanking_tokens_s.xlsx',index = False)
#df_f[df_f['value']>0].to_excel('D://Downloads//comments_tokens.xlsx')
#df.to_excel('D://Downloads//comments_cleaned.xlsx')

In [ ]:
#df_f[df_f['CON_ROW_ID'] ==60427536]
